# Routine to refactor the scraped data

In [1]:
import pandas as pd

In [2]:
# Criar DataFrame
df = pd.read_csv('product_data.csv')
df.shape

(390, 95)

## Let's delete what we don't need

In [3]:
df = df.loc[df['Tipo de Cabelo'].isna() | (df['Tipo de Cabelo'] == '')]
df = df.loc[df['Tipo de suporte ou fixação'].isna() | (df['Tipo de suporte ou fixação'] == '')]
df = df.loc[df['Desenho ou estampa'].isna() | (df['Desenho ou estampa'] == '')]

In [4]:
df['Nome da marca'] = df['Nome da marca'].fillna(df['Marca'])
df['Nome da marca'] = df['Nome da marca'].fillna(df['Número do modelo'])
df['Tensão'] = df['Tensão'].fillna(df['Voltagem'])
df['Velocidade máxima de rotação'] = df['Velocidade máxima de rotação'].fillna(df['Velocidade máxima de centrifugação'])
df['Número de Peças'] = df['Número de Peças'].fillna(df['Número de produtos'])
df['Número de Peças'] = df['Número de Peças'].fillna(df['Número de unidades'])
df['Número de Peças'] = df['Número de Peças'].fillna(df['Número de peças'])
df['Potência'] = df['Potência'].fillna(df['Potência em watts'])
df['Eficiência'] = df['Eficiência'].fillna(df['Etiqueta Nacional de Eficiência Energética (ENCE)'])
df['Tipo de fonte de energia'] = df['Tipo de fonte de energia'].fillna(df['Combustível'])
df['Material'] = df['Material'].fillna(df['Tipo de material'])
df['Material'] = df['Material'].fillna(df['Composição do material'])
df['Material'] = df['Material'].fillna(df['Material da alça ou do cabo'])
df['Tipo de controles'] = df['Tipo de controles'].fillna(df['Painel de controle'])
df['Tipo de controles'] = df['Tipo de controles'].fillna(df['Entrada de usuário'])
df['Tipo de instalação'] = df['Tipo de instalação'].fillna(df['Forma de instalação'])
df['Certificação'] = df['Certificação'].fillna(df['Registro no Inmetro'])
df['Garantia do fabricante'] = df['Garantia do fabricante'].fillna(df['Descrição da garantia'])
df['Número da Peça'] = df['Número da Peça'].fillna(df['Número do modelo'])
df['Informações do modelo'] = (df['Informações do modelo'].fillna(df['Número da Peça'])).str.lower()

In [5]:
to_delete = ['Ranking dos mais vendidos','Tipo de Cabelo','Tipo de suporte ou fixação',
             'Desenho ou estampa','Marca','Baterias inclusas','Baterias inclusas?',
             'Departamento','Funciona a bateria ou pilha?','Funciona com baterias',
             'Precisa de pilhas ou baterias?','Voltagem','Velocidade máxima de centrifugação',
             'Número de peças', 'Número de produtos', 'Número de unidades',
             'Potência em watts', 'Etiqueta Nacional de Eficiência Energética (ENCE)','Combustível',
             'Tipo de material', 'Composição do material', 'Material da alça ou do cabo',
             'Quantidade','Quantidade por pacote', 'Painel de controle',
             'Entrada de usuário','Nome do produto','Unidades',
             'Fator de forma','Dispositivos compatíveis', 'Ano',
             'Formato','Dobradiças da porta','Tipo de acabamento',
             'Correspondência de tamanho','Forma de instalação', 'Contém líquido?',
             'É necessária montagem','Acabamento','Nível de ruído',
             'Altura do cano', 'Tipo de instalação', 'Tipo de queimadores',
             'Peças para montagem','Modelo','Nome do modelo',
             'Estilo', 'Instruções de cuidado','Descrição da garantia',
             'Configuração básica', 'Registro no Inmetro']

In [6]:
df = df.drop(to_delete,axis=1)
df.shape

(382, 44)

## Let's translate what is needed to be translated

In [7]:
# Specify which columns are not going to be translated
col_ignore = ['Link', 'Price', 'Nome da marca', 'Informações do modelo', 'Número da Peça','Fabricante','Avaliações de clientes','Review Count','Image Link' ]

In [8]:
# This function adds GOOGLETRANSLATE({x},'auto') in the objects
def apply_googletranslate(x):
    if pd.isna(x):
        return ''  # If nan shouldn't be translated
    if isinstance(x, (int, float)):
        return x  # If numeric, shouldn't be translated
    return f'=GOOGLETRANSLATE("{x}", "auto")'

# we will not change the original document
df_translated = df.copy()

for col in df.columns:
    if col not in col_ignore:#ignore those we don't want to trnaslate
        df_translated[col] = df[col].apply(apply_googletranslate)

translated_columns = [apply_googletranslate(col) for col in df.columns]
df_translated.columns = translated_columns

# Exibir o DataFrame modificado
print("DataFrame translated:")
print(df_translated)

# Exportar o DataFrame para um arquivo CSV
df_translated.to_csv('final_product_data.csv', index=False, encoding='utf-8')



DataFrame translated:
                      =GOOGLETRANSLATE("Link", "auto")  \
0    https://www.amazon.com.br/sspa/click?ie=UTF8&s...   
1    https://www.amazon.com.br/sspa/click?ie=UTF8&s...   
2    https://www.amazon.com.br/Lava-Programas-Inver...   
3    https://www.amazon.com.br/Secadora-Electrolux-...   
4    https://www.amazon.com.br/Smart-Intelig%C3%AAn...   
..                                                 ...   
384  https://www.amazon.com.br/lavadora-secadora-es...   
386  https://www.amazon.com.br/sspa/click?ie=UTF8&s...   
387  https://www.amazon.com.br/sspa/click?ie=UTF8&s...   
388  https://www.amazon.com.br/sspa/click?ie=UTF8&s...   
389  https://www.amazon.com.br/sspa/click?ie=UTF8&s...   

                      =GOOGLETRANSLATE("Name", "auto")  \
0    =GOOGLETRANSLATE("Mini Lavadora Portátil Para ...   
1    =GOOGLETRANSLATE("Lava e Seca 10,5kg Midea Hea...   
2    =GOOGLETRANSLATE("Lava e Seca 16 Programas Eco...   
3    =GOOGLETRANSLATE("Secadora de Roupas Electro

## Comparing results

In [9]:
df.shape

(382, 44)